# Multi-device Programming

This lab is intended for C/C++ programmers. If you prefer to use Fortran, click [this link.](../Fortran/Lab9_Fortran.ipynb)

---
## Introduction

Many modern machines are now coming equipped with multiple parallel devices. Additionally, there are also small-large networks of interconnected machines that we can program for. To give an idea of what these machines look like, we can look at (currently as of the making of these lab activities) the largest supercomputer in the world, [Summit from Oak Ridge National Lab.](https://www.olcf.ornl.gov/summit/) This is a machine that connects thousands of individual machines (referred to as nodes, in this context), and each node has 6 NVIDIA GPUs. So the goal of this lab is to give you the skills you would need to program for a machine like Summit.

![Multi Device](../images/Multi_Device.png)

There are a few ways that you can use multiple devices with OpenACC:

* Use OpenACC async to asynchronously launch work to multiple devices
* Use OpenMP to launch multiple OpenMP threads and allow each thread to use a device
* Use MPI for distributed programming, splitting the devices across MPI ranks

We will visit each of these individually, and complete an exercise for each.

---

## Multi-Device with OpenACC Async

In a previous lab, we have used the async clause to launch multiple tasks on our device for asynchronous execution. Now, we will use the async clause to launch tasks on more than one device. Let's see an example:

```
// Loop 1
#pragma acc parallel loop
for(int i = 0; i < N; i++) {
    A[i] = max(A[i], 0);
}

// Loop 2
#pragma acc parallel loop
for(int i = 0; i < M; i++) {
    B[i] = max(B[i], 0);
}
```

Our goal is to run Loop 1 on one of our devices, and Loop 2 on another. Let's first go over everything we need to know to make this happen.

### Finding the Number of Available Devices

It is often best to write code to be able to use a variable number of devices. OpenACC allows us to check the number of devices available with a built-in function.

**acc_get_num_devices( devicetype )**

The device type is which kind of device you want to find. For this lab, and in most cases, it is sufficient to use **acc_device_default**.

### Switching to a Different Device

Our devices will be numbered from 0 -> num_devices-1. The active device is default to device0. We can change that with this built-in function:

**acc_set_device_num( devicenum, devicetype )**

We will again use acc_device_default for the device type, and for our code example we will use device0 and device1.

### OpenACC async

This was covered in an earlier lab, but let's do a quick recap. Normal OpenACC behavior is for the host thread to pause and wait for all OpenACC regions to finish. This could be any of the parallel, kernels, enter data, exit data, or update directives. If we add the async clause to any of these directives, we tell the host thread **not to wait**. This means that we can start a parallel loop on a device, and, without waiting, continue on to any other unrelated host code. We will use this feature to launch a kernel on one device asynchronously. Then, switch to a different device and launch another kernel. 

### A Simple Multi-Device Code

```
int ndevice = acc_get_num_devices(acc_device_default);

acc_set_device_num( 0, acc_device_default );

// Loop 1
#pragma acc parallel loop async
for(int i = 0; i < N; i++) {
    A[i] = max(A[i], 0);
}

acc_set_device_num( 1, acc_device_default );

// Loop 2
#pragma acc parallel loop async
for(int i = 0; i < M; i++) {
    B[i] = max(B[i], 0);
}
```

### Special Cases

Multi-device with async is the least straightforward of our three methods. There are a few things that we have to take special care with for it to work properly.

1) Adding async to an enter/exit data might not always work the way we want it to. Our host thread may have trouble making enter/exit data truly asynchronous because of the allocation/deallocation it has to do. Because of this, it is recommended to do device allocation/deallocation synchronously at the beginning and end, and use asynchronous updates whenever we need to move data.

```
int ndevice = acc_get_num_devices(acc_device_default);

for(int n = 0; n < ndevices; n++) {
    acc_set_device_num( n, acc_device_default);
    #pragma acc enter data create( ... )
}

// Multi-device Code
#pragma acc update self( ... ) async

for(int n = 0; n < ndevices; n++) {
    acc_set_device_num( n, acc_device_default);
    #pragma acc exit data delete( ... )
}
```

2) One important aspect of using OpenACC async is waiting at certain points of the code for synchronization. This is accomplished with the **wait** directive. There is not a way to wait for all devices in a single directive, so whenever we do synchronization, we must loop through all devices, and wait for each.

```
for(int n = 0; n < ndevices; n++) {
    #pragma acc parallel loop async
    // Loop Code
}

for(int n = 0; n < ndevices; n++) {
    #pragma acc wait
}
```

---

### A More Realistic Code Example

The code we looked at earlier was far too basic to be applicable to most real-world applications. Let's look at a slightly more complex code called Mandelbrot. If you have followed along with the lecture material, you may be familar with the Madelbrot code. It is an image processing code that creates a "mandelbrot" image. The code is somewhat straightfoward. We are creating an image a single pixel at a time, and each pixel is independent of eachother. Our goal for making it multi-device is to break the image creation into multiple blocks, and to compute each block on a different device. Here is the code:

```
const int w = 4098;
const int h = 4098;

int ndevices = acc_get_num_devices( acc_device_default );

int rowsPerBlock = h / ndevices;
int blockSize    = w*h / ndevices;

for(int n = 0; n < ndevices; n++) {
    acc_set_device_num( n, acc_device_default );
    #pragma acc enter data create(outImage[n*rowsPerBlock:blockSize])
}

for(int n = 0; n < ndevices; n++) {
    acc_set_device_num( n, acc_device_default );
    #pragma acc parallel loop async
    for(int y = n*rowsPerBlock; y < (n+1)*rowsPerBlock; y++) {
        for(int x = 0; x < w; x++) {
            outImage[y*w + x] = mandelbrot(x, y);
        }
    }
    #pragma acc update self(outImage[n*rowsPerBlock:blockSize]) async
}

for(int n = 0; n < ndevices; n++) {
    acc_set_device_num( n, acc_device_default );
    #pragma acc wait
    #pragma acc exit data delete(outImage[n*rowsPerBlock:blockSize])
}
```

This code combines all the techniques discussed previously, and is a good template for the code we will work on in this lab. The code we have is an image filtering code. If you have also completed the OpenACC Async lab, then you should be familar with this code. Run it below to get a baseline performance.

In [ ]:
!make clean -C Async && make -C Async

Your goal is to implement all the techniques displayed in the Mandelbrot code. This includes getting the number of devices, handling all of the device allocation/deallocation, and launching the loops and updates asynchronously on multiple devices. The code is found in [filter.c](/edit/C/Async/filter.c). Edit it to allow for multiple devices, and run the code below to test your changes.

In [ ]:
!make clean -C Async && make -C Async

If you want to check your answers, you can see our solution [here.](/edit/C/Async/Solution/filter.c)

---

## Multi-Device with OpenMP + OpenACC

The main benefit of using OpenMP over Async is that OpenMP gives us several threads to work with, rather than a single host thread. Each thread can use its own device, and we do not have to worry about the same synchronization problems as before. You do not need to be very familar with OpenMP to do multi-device programming; you will only need to learn some basic OpenMP loop syntax.

![OpenMP](../images/OpenMP.png)

There are three primary strategies we can use OpenMP to achieve multi-device programming:

1) Create a loop that goes over all devices and parallelize that loop with OpenMP. This is a similar strategy that we used when doing multi-device with async. Here is an example:

```
int ndevices = acc_get_num_devices( acc_device_default );

#pragma omp parallel for
for(int n = 0; n < ndevices; n++) {
    acc_set_device_num( n, acc_device_default );
    
    #pragma acc parallel loop
    for( ... ) // Notice that async is no longer needed
}
```

2) Break the code into however many blocks we desire. Parallelize those blocks with OpenMP, and be sure to set the number of OpenMP threads to match the number of devices. Then, each OpenMP thread will use a single device to compute a portion of the total blocks. Here is an example:

```
int ndevices = acc_get_num_devices( acc_device_default );
int nblocks  = 32;

#pragma omp parallel for num_threads(ndevices)
for(int n = 0; n < nblocks; n++) {
    int tid = omp_get_thread_num();
    acc_set_device_num( tid, acc_device_default );
    
    #pragma acc parallel loop
    for( ... )
}
```

3) Create an OpenMP parallel region, setting the number of OpenMP threads to match the number of devices. Give each thread a single device, and run whatever computation you need. This is similar to the previous example, but we only need to call **acc_set_device_num** once per thread.

```
int ndevices = acc_get_num_devices( acc_device_default );

#pragma omp parallel num_threads(ndevices)
{
    int tid = omp_get_thread_num();
    acc_set_device_num( tid, acc_device_default );
    
    #pragma acc parallel loop
    for( ... ) {
        < Loop Code >
    }
}
```

We can also use this to apply the code blocking similar to strategy 2:

```
int ndevices = acc_get_num_devices( acc_device_default );
int nblocks = 32;

#pragma omp parallel num_threads(ndevices)
{
    int tid = omp_get_thread_num();
    acc_set_device_num( tid, acc_device_default );
    
    #pragma omp for
    for( int block = 0; block < nblocks; block++ ) {
        #pragma acc parallel loop
        for( ... ) {
            < Loop Code >
        }
    }
}
```

### Madelbrot Example and Exercise

To prepare for applying OpenMP to our code, let's look at the Mandelbrot example again (using strategy 3).

```
const int w = 4098;
const int h = 4098;

int ndevices = acc_get_num_devices( acc_device_default );
int nblocks = 32;

int rowsPerBlock = h / nblocks;
int blockSize    = w*h / nblocks;

#pragma omp parallel num_threads(ndevices)
{
    int tid = omp_get_thread_num();
    acc_set_device_num( tid, acc_device_default );

    #pragma omp for
    for(int n = 0; n < nblocks; n++) {
    
        #pragma acc parallel loop copyout(outImage[n*rowsPerBlock:blockSize])
        for(int y = n*rowsPerBlock; y < (n+1)*rowsPerBlock; y++) {
            for(int x = 0; x < w; x++) {
                outImage[y*w + x] = mandelbrot(x, y);
            }
        } // End OpenACC parallel loop
        
    } // End OpenMP block loop
} // End OpenMP parallel region
```

Now, let's apply one of these strategies (or try all of them if you want) to our image filtering code. Run the code below and get some baseline performance.

In [ ]:
!make clean -C OpenMP && make -C OpenMP

Now, edit the code however you like [here.](/edit/C/OpenMP/filter.c) Run the code below to check your results.

In [ ]:
!make clean -C OpenMP && make -C OpenMP

If you would like to see our solution, you may do here [here.](/edit/C/OpenMP/Solution/filter.c)

---

## Multi-Device with MPI + OpenACC

This is by far the most practical use of multi-device with OpenACC. Many codes are already parallelized with MPI for CPUs, and OpenACC is meant to be easy to implement on-top-of a MPI code. This section will require a more high-level understanding of MPI than the OpenMP code did.

MPI+OpenACC allows us to connect multiple devices across multiple nodes on a network. Here is a visualization of what we hope to accomplish:

![MPI](../images/MPI.png)

The basic premise of a MPI+OpenACC code is that we have a MPI code (with distributed memory and communcation), and within that code we accelerate our computationally intensive code using OpenACC like normal. We can assign each MPI rank its own GPU, or we can allow multiple ranks to share a GPUs (though this is usually not beneficial). Let's look at a simple code example:

```
int main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);
    int nranks, rank;
    MPI_Comm_size(MPI_COMM_WORLD, &nranks);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    int ndevices = acc_get_num_devices(acc_device_default);
    acc_set_device_num(rank%ndevices, acc_device_default);
    
    if(rank == 0) {
        #pragma acc parallel loop
        for( ... ) {
            < Rank 0 Loop >
        }
    } else if(rank == 1) {
        #pragma acc parallel loop
        for( ... ) {
            < Rank 1 Loop >
        }
    }
    
    MPI_Finalize();
```

This example is much more basic than the codes you will most likely work with. So let's look again at our mandelbrot code.

### Mandelbrot Example

```
MPI_Init(&argc, &argv);
int nranks, rank;
MPI_Comm_size(MPI_COMM_WORLD, &nranks);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
int ndevices = acc_get_num_devices(acc_device_default);
acc_set_device_num(rank%ndevices, acc_device_default);

const int w = 4098;
const int h = 4098;

int rowsPerRank = h / nranks;
int pixelsPerRank = (w*h) / nranks;
int yStart = rank*rowsPerRank;
int yEnd   = yStart + rowsPerRank;

#pragma acc parallel loop copyout(outData[yStart*w:pixelsPerRank)
for(int y = yStart; y < yEnd; y++) {
    for(int x = 0; x < w; x++) {
        outData[y*w + x] = mandelbrot(x, y);
    }
}

// MPI_Gather syntax ***************************************************
// MPI_Gather(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
//            void *recvbuf, int recvcount, MPI_Datatype recvtype,
//            int root, MPI_Comm comm)

MPI_Gather(&outData[yStart*w], pixelsPerRank, MPI_UNSIGNED_INT, // Send info
           outData, pixelsPerRank, MPI_UNSIGNED_INT,            // Recv info
           0, MPI_COMM_WORLD);
    
MPI_Finalize();
```

If you are more savvy with MPI, you might notice that a simple MPI_Gather here is somewhat lazy. It would be more correct to use MPI_Gatherv to account for image sizes that are not evenly divisible by the number of ranks. You may include this in your code, and our solution will also have it, but this lab is designed so that if you are less familar with MPI you do not need to worry about it to complete it.

### Image Filter Exercise

Using mandelbrot as an example, let's parallelize our image filter code with MPI. We will start with a basic OpenACC (single device) implementation, and you will need to add all of the MPI interface to the code. Let's first run the code and get a baseline.

We have already added everything needed to [main.c](/edit/C/MPI/main.c), so you do not need to edit this file. Instead, edit [filter.c](/edit/C/MPI/filter.c) and add the **acc_set_device_num** and the rest of the MPI code. We will post some helpful MPI syntax below, and some general tips:

* You will need to distribute the input array across the MPI ranks. If you are unfamilar with MPI, we recommend using an MPI_Bcast to transer the entire input array to each rank. If you are more familiar with MPI, we recommend you use MPI_Scatterv to only transfer the data you need from the input array.
* You will need to edit the Y loop to compute a subset of the rows based on MPI rank.
* You will need to gather all of the output arrays at the end.

MPI Syntax: \*note: for the MPI_Comm use the MPI_COMM_WORLD constant, for the MPI_Datatype use MPI_UNSIGNED_CHAR, and for root use 0\*

```
MPI_Bcast( void \*buffer, int count, MPI_Datatype datatype, int root, MPI_Comm comm )  
MPI_Gather(const void \*sendbuf, int sendcount, MPI_Datatype sendtype, 
           void \*recvbuf, int recvcount, MPI_Datatype recvtype, 
           int root, MPI_Comm comm)  
MPI_Scatterv(const void \*sendbuf, const int \*sendcounts,   
               const int \*displs, MPI_Datatype sendtype,   
             void \*recvbuf, int recvcount, MPI_Datatype recvtype,   
             int root, MPI_Comm comm)
```
             


In [ ]:
!make clean -C MPI && make -C MPI

If you would like to see our solution, you may do here [here.](/edit/C/MPI/Solution/filter.c)

---

## Bonus Activity, Device-to-Device MPI Data Transfers

We can transfer data directly from device-to-device when calling MPI functions. We have to use a strategy covered in an earlier Lab; the OpenACC host_data directive. By using the **host_data** directive with the **use_device** clause, we can pass device pointers directly to our MPI function calls. And if your machine supports it, this will allow MPI to transfer data directly between devices. Here is an example:

```
#pragma acc host_data use_device(A)
{
    MPI_Gather(A + offset, size, MPI_INT, outData, size, MPI_INT, 0, MPI_COMM_WORLD);          
}
```

If you would like to test it out, go back to the code you wrote for this lab, and trying adding the host_data directive to your MPI calls. [Here is a shortcut to filter.c.](/edit/C/filter.c)

In [ ]:
!make clean -C MPI && make -C MPI

---

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well.

You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
rm -f openacc_files.zip
zip -r openacc_files.zip *

**After** executing the above zip command, you should be able to download the zip file [here](files/openacc_files.zip)